# Ocean interior carbon storage: Changes in preformed vs regenerated carbon

In [23]:
from dask.distributed import Client

client = Client("tcp://10.32.15.116:35423")
client

Client Scheduler: tcp://10.32.15.116:35423 Dashboard: /user/0000-0002-2993-7484/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 11.50 GB


In [1]:
%matplotlib inline

import xarray as xr
import intake
import numpy as np
from cmip6_preprocessing.preprocessing import read_data
import warnings

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
# Functions
def calc_o2sat_gsw(practical_salinity,potential_temperature,latitude,longitude,pressure,dask_permission='allowed'):
    import gsw
    
    #Calculate Absolute Salinity
    gsw_as=xr.apply_ufunc(gsw.SA_from_SP,practical_salinity,pressure,longitude,latitude,dask=dask_permission)
    
    #Calculate Conservative Temperature
    gsw_ct=xr.apply_ufunc(gsw.CT_from_t,gsw_as,potential_temperature,pressure,dask=dask_permission)

    #Calculate Oxygen Saturation
    gsw_o2sat=xr.apply_ufunc(gsw.O2sol,gsw_as,gsw_ct,pressure,longitude,latitude,dask=dask_permission)*1.0245e-3 # Convert umol/kg to mol/m3

    return gsw_o2sat

In [3]:
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")

In [8]:
import pprint 
uni_dict = col.unique(['source_id', 'experiment_id', 'table_id'])
#pprint.pprint(uni_dict, compact=True)

{'experiment_id': {'count': 29,
                   'values': ['ssp370', 'esm-ssp585', '1pctCO2-bgc', 'hist-bgc',
                              '1pctCO2', 'abrupt-4xCO2', 'historical',
                              'piControl', 'amip', 'esm-hist', 'esm-piControl',
                              'hist-GHG', 'hist-aer', 'hist-nat', 'dcppA-assim',
                              'dcppA-hindcast', 'dcppC-hindcast-noAgung',
                              'dcppC-hindcast-noElChichon',
                              'dcppC-hindcast-noPinatubo', 'highresSST-present',
                              'control-1950', 'hist-1950', 'deforest-globe',
                              'esm-ssp585-ssp126Lu', 'omip1', 'lgm', 'ssp126',
                              'ssp245', 'ssp585']},
 'source_id': {'count': 48,
               'values': ['BCC-ESM1', 'BCC-CSM2-MR', 'CanESM5', 'CNRM-ESM2-1',
                          'UKESM1-0-LL', 'GISS-E2-1-G', 'CESM2', 'GFDL-ESM4',
                          'AWI-CM-1-1-MR', 'CAM

In [28]:
models = set(uni_dict['source_id']['values']) # all the models

for experiment_id in ['historical']:
    query = dict(experiment_id=experiment_id, 
                 grid_label='gn',
                 table_id='Oyr',
                 variable_id=['o2','thetao','so','dissic'])  
    cat = col.search(**query)
    models = models.intersection({model for model in cat.df.source_id.unique().tolist()})

# for oxygen, ensure the CESM2 models are not included (oxygen was erroneously submitted to the archive)
# UKESM has an issue with the attributes
models = models - {'CESM2-WACCM', 'CESM2'}
models = list(models)
models

['MIROC-ES2L', 'IPSL-CM6A-LR', 'CanESM5']

In [29]:
# read all data with volume and oxygen
with warnings.catch_warnings(): # these lines just make sure that the warnings dont clutter your notebook
    warnings.simplefilter("ignore")
    data_dict_thk = read_data(col,
                    experiment_id=['historical'],
                    grid_label='gn',
                    variable_id=['o2','dissic','thetao','so'],
                    table_id = ['Oyr'],
                    source_id = models,
                    #member_id = 'r1i1p1f1', # so that this runs faster for testing
                    required_variable_id = ['o2','dissic','thetao','so']
                    )

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 3 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 0 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 0 group(s)


In [30]:
data_dict_thk

{}